In [238]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import requests

In [239]:
import requests

# Backend API URL
url = "http://127.0.0.1:5000/survey"  # Replace 'endpoint' with your actual route

try:
    # Make a GET request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data1 = response.json()
        print("Data fetched successfully:", data1)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        print("Response:", response.text)
except Exception as e:
    print("Error occurred:", e)

Error occurred: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /survey (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000192FB672D10>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


In [240]:
class Model(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(Model, self).__init__()
        layers = []
        in_dim = input_dim
        for h in hidden_dims:
            layer = nn.Linear(in_dim, h)
            torch.nn.init.normal_(layer.weight)
            layers.append(layer)
            layers.append(nn.ReLU())
            in_dim = h
        final_layer = nn.Linear(in_dim, output_dim)
        torch.nn.init.normal_(final_layer.weight)
        layers.append(final_layer)
        layers.append(nn.Softmax(dim=1))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [241]:
input_dim = 5
hidden_dims = [5, 5]
output_dim = 2

In [242]:
model0 = Model(input_dim = input_dim, hidden_dims = hidden_dims, output_dim = output_dim)

In [243]:
state_dict = torch.load('model0.pth')  # Replace with the actual file
model0.eval()
model0.load_state_dict(state_dict)

<All keys matched successfully>

In [244]:
data1['survey'][0]

{'age': 6,
 'dad': 1,
 'gender': 1,
 'id': 1,
 'mom': 1,
 'name': 'Y Niteesh Reddy',
 'time_reading': 0,
 'time_sports': 4,
 'time_studying': 1,
 'time_tv': 7,
 'time_video_games': 1}

In [245]:
compositehr = 3*(data1['survey'][0]['time_reading']+data1['survey'][0]['time_studying'])+2*data1['survey'][0]['time_video_games']+data1['survey'][0]['time_tv']

In [246]:
compositehr

12

In [247]:
if data1['survey'][0]['gender'] == 'Male':
    data1['survey'][0]['gender'] = 0
else:
    data1['survey'][0]['gender'] = 1

In [248]:
input_df = []
input_df.append(data1['survey'][0]['age'])
input_df.append(data1['survey'][0]['gender'])
input_df.append(compositehr)
input_df.append(data1['survey'][0]['mom'])
input_df.append(data1['survey'][0]['dad'])
input = torch.tensor(input_df)

In [249]:
float_input

tensor([[ 6.,  1., 12.,  1.,  1.]])

In [250]:
with torch.no_grad():
    float_input = input.float()
    float_input = float_input.unsqueeze(0)
    output = model0(float_input)

In [251]:
int(output[0][0])

0

In [252]:
#risk_float = 0.1*output[0][0] + 0.3*data['tests'][0]['snell'] + 0.3*data['tests'][0]['asti'] + 0.3*data['tests'][0]['blur']

In [253]:
risk = {
    'sol' : float(output[0][0])
}

In [254]:
import requests

# Backend API URL
url = "http://127.0.0.1:5000/result"  # Replace with your actual endpoint

try:
    # Making the POST request
    response = requests.post(url, json=risk)  # Use `json` to send a JSON payload

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data1 = response.json()
        print("Data fetched successfully:", data1)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        print("Response:", response.text)
except Exception as e:
    print("Error occurred:", e)


Error occurred: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /result (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000192FBFAE310>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
